In [ ]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install minsearch

In [44]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [ ]:
import minsearch
import json

with open('documents.json', 'rt') as f_in:
    docs_raw =json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "selection"],
    keyword_fields=["course"]
)

index.fit(documents)

In [43]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [52]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [53]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [56]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [57]:
quesion = "how do I run kafka?"
rag(quesion)

'To run Kafka, if you are using Java Kafka, you should navigate to the project directory and execute the following command in your terminal:\n\n```shell\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you encounter a "ModuleNotFoundError" while running a Python Kafka producer, consider creating a virtual environment and running the required scripts and installing dependencies as outlined in the provided solution. You may also need to replace the Kafka dependency with `kafka-python-ng` as per the latest recommendation.'